In [3]:
# #Go through the ECG dataset and append ID (index) and time variable. Transpose to long format dataset for tsfresh. Endproduct
# # is a long format dataset of ECG arrays
import numpy as np
import pandas as pd

ecg_np_to_extract= np.load('selected_ecg_downsampled_final.npy')

final=pd.DataFrame()
i = 0

for row in ecg_np_to_extract:
        arrleads = np.transpose(row)
        df = pd.DataFrame(arrleads)
        dft = np.transpose(df)
        dft['Time'] = np.arange(len(dft)) #Time label
        dft['ID'] = i #ID
        i += 1
        final=final.append(dft,ignore_index=True)

final.to_csv('final_transposed_leads_distant_downsampled.csv')

C:\Users\mzkolk\AppData\Local\Temp/ipykernel_13012/4038786709.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final=final.append(dft,ignore_index=True)


In [28]:
print(final_dr.ID.unique())
print(final_dist.ID.unique())

[1799 1800 1801 ... 2939 2940 2941]
[   0    1    2 ... 1796 1797 1798]


In [1]:
# Load DR long format ECG
import pandas as pd
final_dr = pd.read_csv('final_transposed_leads_deep_risk.csv')
final_dr['ID'] = final_dr['ID'] + 1798
final_dr = final_dr.rename(columns={'0': '0', '0.1': '1', '0.2' : '2', 
                                    '0.3' : '3', '0.4' : '4',                  
                                    '0.5' : '5', '0.6' : '6', '0.7' : '7',
                                   '0.8' : '8', '0.9' : '9', '0.10' : '10', '0.11' : '11'})
#Load DIST long format ECG
final_dist = pd.read_csv('final_transposed_leads_distant_downsampled.csv')

#Concatenate both DIST and DR cohorts
frames = [final_dist, final_dr]
ecg_both_sets = pd.concat(frames)
ecg_both_sets.to_csv('ecg_both_sets.csv')

In [37]:
df = ecg_both_sets
df.drop("Unnamed: 0", axis=1, inplace=True)

#Run tfresh
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
efficientParametersForTsFresh = EfficientFCParameters()
extracted_featuresefficient = extract_features(df, column_id="ID", column_sort="Time", default_fc_parameters=efficientParametersForTsFresh)

#Impute missings values
from tsfresh.utilities.dataframe_functions import impute
extracted_featuresefficient_impute = impute(extracted_featuresefficient)

#Save with imputed missings values
extracted_featuresefficient.to_csv('extracted_featuresefficient.csv')

Feature Extraction: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [5:12:23<00:00, 1874.35s/it]
c:\users\mzkolk\pycharmprojects\pythonproject\venv\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['0__query_similarity_count__query_None__threshold_0.0'
 '1__query_similarity_count__query_None__threshold_0.0'
 '2__query_similarity_count__query_None__threshold_0.0'
 '3__query_similarity_count__query_None__threshold_0.0'
 '4__query_similarity_count__query_None__threshold_0.0'
 '5__query_similarity_count__query_None__threshold_0.0'
 '6__query_similarity_count__query_None__threshold_0.0'
 '7__query_similarity_count__query_None__threshold_0.0'
 '8__query_similarity_count__query_None__threshold_0.0'
 '9__query_similarity_count__query_None__threshold_0.0'
 '10__query_similarity_count__query_None__threshold_0.0'
 '11__query_similarity_count__query_None__threshold_0.0'] did not h

In [51]:
test = extracted_featuresefficient
test['Python_ID_Index'] = test.reset_index().index
extracted_featuresefficient.to_csv('extracted_featuresefficient_withID.csv')